task name: Resolve FastTreeWithOptions task
task description: Implement the provided C# code as a dotnet interactive notebook

In [ ]:
#r "nuget: Microsoft.ML"
#r "nuget: Microsoft.ML.FastTree"

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers.FastTree;

In [ ]:
var mlContext = new MLContext(seed: 0);

In [ ]:
// Class used to capture data points.
private class DataPoint
{
    [KeyType(5)]
    public uint Label { get; set; }
    [KeyType(100)]
    public uint GroupId { get; set; }
    [VectorType(50)]
    public float[] Features { get; set; }
}

private static IEnumerable<DataPoint> GenerateRandomDataPoints(int count,
    int seed = 0, int groupSize = 10)
{
    var random = new Random(seed);
    float randomFloat() => (float)random.NextDouble();
    for (int i = 0; i < count; i++)
    {
        var label = random.Next(0, 5);
        yield return new DataPoint
        {
            Label = (uint)label,
            GroupId = (uint)(i / groupSize),
            Features = Enumerable.Repeat(label, 50).Select(
                x => randomFloat() + x * 0.1f).ToArray()
        };
    }
}

var dataPoints = GenerateRandomDataPoints(1000);
var trainingData = mlContext.Data.LoadFromEnumerable(dataPoints);

In [ ]:
var options = new FastTreeRankingTrainer.Options
{
    EarlyStoppingMetric = EarlyStoppingRankingMetric.NdcgAt3,
    FeatureFirstUsePenalty = 0.1,
    NumberOfTrees = 50,
    RowGroupColumnName = "GroupId"
};

var pipeline = mlContext.Ranking.Trainers.FastTree(options);
var model = pipeline.Fit(trainingData);

In [ ]:
var testData = mlContext.Data.LoadFromEnumerable(GenerateRandomDataPoints(500, seed: 123));
var transformedTestData = model.Transform(testData);

In [ ]:
private class Prediction
{
    public uint Label { get; set; }
    public float Score { get; set; }
}

public static void PrintMetrics(RankingMetrics metrics)
{
    Console.WriteLine("DCG: " + string.Join(", ",
        metrics.DiscountedCumulativeGains.Select(
            (d, i) => (i + 1) + ":" + d + ":F2").ToArray()));
    Console.WriteLine("NDCG: " + string.Join(", ",
        metrics.NormalizedDiscountedCumulativeGains.Select(
            (d, i) => (i + 1) + ":" + d + ":F2").ToArray()));
}

var topTransformedTestData = mlContext.Data.TakeRows(transformedTestData, 5);
var predictions = mlContext.Data.CreateEnumerable<Prediction>(topTransformedTestData, reuseRowObject: false).ToList();
foreach (var p in predictions)
    Console.WriteLine("Label: " + p.Label + ", Score: " + p.Score);

var metrics = mlContext.Ranking.Evaluate(transformedTestData);
PrintMetrics(metrics);